In [595]:
import json
import altair as alt
from altair import expr, datum
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests

In [596]:
import colorsys
from matplotlib.colors import to_hex, to_rgb


def scale_lightness(rgb, scale_l):
    rgbhex = False
    if "#" in rgb:
        rgb = to_rgb(rgb)
        rgbhex = True
    # convert rgb to hls
    h, l, s = colorsys.rgb_to_hls(*rgb)
    # manipulate h, l, s values and return as rgb
    c = colorsys.hls_to_rgb(h, min(1, l * scale_l), s=s)
    if rgbhex:
        c = to_hex(c)
    return c

In [597]:
LOCAL = True

if LOCAL:
    local_suffix = "_local"
else:
    local_suffix = ""

In [598]:
%%capture pwd
!pwd

In [599]:
# uid = "2021-05-05-which-firms-and-industries-have-been-most-affected-by-covid-update"  # article unique ID
uid = pwd.stdout.split("/")[-1].split("\r")[0]
eco_git_home = (
    "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/"
)
eco_git_path = eco_git_home + "articles/" + uid + "/data/"
vega_embed = requests.get(eco_git_home + "guidelines/html/vega-embed.html").text
colors = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-colors.json").content
)
category_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-category-color.json").content
)
hue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-single-hue-color.json").content
)
mhue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-multi-hue-color.json").content
)
div_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-diverging-color.json").content
)
config = json.loads(
    requests.get(eco_git_home + "guidelines/charts/eco-global-config.json").content
)
height = config["height"]
width = config["width"]
uid, height, width

('2021-06-30-how-has-the-pandemic-affected-household-finances-in-developing-economies',
 300,
 500)

# Fig 1

In [600]:
df = (
    pd.read_csv(
        "raw/Fig1.csv",
    )
    .dropna(how="all")
    .replace("Emerging economies / E. Europe", "Emerging economies / Eastern Europe")
)

In [601]:
f = "fig1_participation"
f1 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f1.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f1 = df
df.head()

,Group,Value,Variable
0,Developed economies,0.404055,Debt repayment
1,Emerging economies / Eastern Europe,0.118843,Debt repayment
2,Emerging economies / Eastern Europe,0.999884,Food &\r\nutilities
3,Developed economies,0.998759,Food &\r\nutilities
4,Emerging economies / Eastern Europe,0.129638,Rent


In [602]:
base = (
    alt.Chart(f1)
    .encode(tooltip="tooltip:N")
    .transform_calculate(tooltip="datum.Group+' 📈 '+round(datum.Value*100)+'%'")
)
selector1 = [
    "Physical assets\r\n(incl. real estate)",
    "Illiquid fin. assets",
    "Liquid fin. assets",
]
base1 = base.transform_filter(
    alt.FieldOneOfPredicate(field="Variable", oneOf=selector1)
)
bars1 = (
    base1.mark_bar(height=15, opacity=0.9, color=colors["eco-turquiose"], yOffset=-8)
    .encode(x="Value:Q", y=alt.Y("Variable:N"))
    .transform_filter("datum.Group=='Developed economies'")
)
bars2 = (
    base1.mark_bar(height=15, opacity=0.9, color=colors["eco-yellow"], yOffset=8)
    .encode(
        x=alt.X(
            "Value:Q",
            axis=alt.Axis(
                grid=False,
                title="",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                format=".0%",
                tickCount=10,
                orient="bottom",
                labelAngle=0,
            ),
        ),
        y=alt.Y(
            "Variable:N",
            axis=alt.Axis(
                grid=False,
                title="Assets",
                titleX=0,
                titleY=-5,
                titleBaseline="bottom",
                titleAngle=0,
                titleAlign="left",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
            ),
        ),
    )
    .transform_filter("datum.Group=='Emerging economies / Eastern Europe'")
)
text1 = bars1.mark_text(dx=-3, dy=-7, align="right").encode(
    text=alt.Text("Value:Q", format=".0%"),
    color=alt.condition(
        datum.Value < 0.08,
        alt.ColorValue(None),
        alt.ColorValue("white"),
    ),
)
text2 = bars2.mark_text(dx=-3, dy=9, align="right").encode(
    text=alt.Text("Value:Q", format=".0%"),
    color=alt.condition(
        datum.Value < 0.08,
        alt.ColorValue(None),
        alt.ColorValue("white"),
    ),
)
labels1 = (
    bars1.mark_text(
        dx=3, dy=-7, align="left", color=scale_lightness(colors["eco-turquiose"], 0.8)
    )
    .encode(
        text="Group:N",
    )
    .transform_filter("datum.Variable=='Illiquid fin. assets'")
)
labels2 = (
    bars2.mark_text(
        dx=3, dy=9, align="left", color=scale_lightness(colors["eco-yellow"], 0.8)
    )
    .encode(
        text="Group:N",
    )
    .transform_filter("datum.Variable=='Illiquid fin. assets'")
)

layer1 = (
    (
        (bars1 + bars2 + text1 + text2 + labels1 + labels2).properties(
            height=alt.Step(37), width=300
        )
    )
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + "a.json")
layer1

alt.LayerChart(...)

In [603]:
base = (
    alt.Chart(f1)
    .encode(tooltip="tooltip:N")
    .transform_calculate(tooltip="datum.Group+' 📈 '+round(datum.Value*100)+'%'")
)
selector2 = [
    "Any debt",
    "Unsecured debt  ",
    "Secured debt",
]
base1 = base.transform_filter(
    alt.FieldOneOfPredicate(field="Variable", oneOf=selector2)
)
bars1 = (
    base1.mark_bar(height=15, opacity=0.9, color=colors["eco-turquiose"], yOffset=-8)
    .encode(x="Value:Q", y=alt.Y("Variable:N"))
    .transform_filter("datum.Group=='Developed economies'")
)
bars2 = (
    base1.mark_bar(height=15, opacity=0.9, color=colors["eco-yellow"], yOffset=8)
    .encode(
        x=alt.X(
            "Value:Q",
            axis=alt.Axis(
                grid=False,
                title="",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                format=".0%",
                tickCount=10,
                orient="bottom",
                labelAngle=0,
            ),
        ),
        y=alt.Y(
            "Variable:N",
            axis=alt.Axis(
                grid=False,
                title="Liabilities",
                titleX=0,
                titleY=-5,
                titleBaseline="bottom",
                titleAngle=0,
                titleAlign="left",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
            ),
        ),
    )
    .transform_filter("datum.Group=='Emerging economies / Eastern Europe'")
)
text1 = bars1.mark_text(dx=-3, dy=-7, align="right").encode(
    text=alt.Text("Value:Q", format=".0%"),
    color=alt.condition(
        datum.Value < 0.08,
        alt.ColorValue(None),
        alt.ColorValue("white"),
    ),
)
text2 = bars2.mark_text(dx=-3, dy=9, align="right").encode(
    text=alt.Text("Value:Q", format=".0%"),
    color=alt.condition(
        datum.Value < 0.08,
        alt.ColorValue(None),
        alt.ColorValue("white"),
    ),
)
labels1 = (
    bars1.mark_text(
        dx=3, dy=-7, align="left", color=scale_lightness(colors["eco-turquiose"], 0.8)
    )
    .encode(
        text="Group:N",
    )
    .transform_filter("datum.Variable=='Unsecured debt  '")
)
labels2 = (
    bars2.mark_text(
        dx=3, dy=9, align="left", color=scale_lightness(colors["eco-yellow"], 0.8)
    )
    .encode(
        text="Group:N",
    )
    .transform_filter("datum.Variable=='Unsecured debt  '")
)

layer2 = (
    (
        (bars1 + bars2 + text1 + text2 + labels1 + labels2).properties(
            height=alt.Step(37), width=300
        )
    )
    .configure_view(stroke=None)
    .properties(title="")
)
layer2.save("visualisation/" + f + "b.json")
layer2

alt.LayerChart(...)

In [604]:
base = (
    alt.Chart(f1)
    .encode(tooltip="tooltip:N")
    .transform_calculate(tooltip="datum.Group+' 📈 '+round(datum.Value*100)+'%'")
)
selector3 = [
    "Debt repayment",
    "Food &\r\nutilities",
    "Rent",
    "Other expenses\r\n(incl. net savings)",
]
base1 = base.transform_filter(
    alt.FieldOneOfPredicate(field="Variable", oneOf=selector3)
)
bars1 = (
    base1.mark_bar(height=15, opacity=0.9, color=colors["eco-turquiose"], yOffset=-8)
    .encode(x="Value:Q", y=alt.Y("Variable:N"))
    .transform_filter("datum.Group=='Developed economies'")
)
bars2 = (
    base1.mark_bar(height=15, opacity=0.9, color=colors["eco-yellow"], yOffset=8)
    .encode(
        x=alt.X(
            "Value:Q",
            axis=alt.Axis(
                grid=False,
                title="",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                format=".0%",
                tickCount=10,
                orient="bottom",
                labelAngle=0,
            ),
        ),
        y=alt.Y(
            "Variable:N",
            axis=alt.Axis(
                grid=False,
                title="Consumption",
                titleX=0,
                titleY=-5,
                titleBaseline="bottom",
                titleAngle=0,
                titleAlign="left",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
            ),
        ),
    )
    .transform_filter("datum.Group=='Emerging economies / Eastern Europe'")
)
text1 = bars1.mark_text(dx=-3, dy=-7, align="right").encode(
    text=alt.Text("Value:Q", format=".0%"),
    color=alt.condition(
        datum.Value < 0.08,
        alt.ColorValue(None),
        alt.ColorValue("white"),
    ),
)
text2 = bars2.mark_text(dx=-3, dy=9, align="right").encode(
    text=alt.Text("Value:Q", format=".0%"),
    color=alt.condition(
        datum.Value < 0.08,
        alt.ColorValue(None),
        alt.ColorValue("white"),
    ),
)
labels1 = (
    bars1.mark_text(
        dx=3, dy=-7, align="left", color=scale_lightness(colors["eco-turquiose"], 0.8)
    )
    .encode(
        text="Group:N",
    )
    .transform_filter("datum.Variable=='Rent'")
)
labels2 = (
    bars2.mark_text(
        dx=3, dy=9, align="left", color=scale_lightness(colors["eco-yellow"], 0.8)
    )
    .encode(
        text="Group:N",
    )
    .transform_filter("datum.Variable=='Rent'")
)

layer3 = (
    (
        (bars1 + bars2 + text1 + text2 + labels1 + labels2).properties(
            height=alt.Step(37), width=300
        )
    )
    .configure_view(stroke=None)
    .properties(title="")
)
layer3.save("visualisation/" + f + "c.json")
layer3

alt.LayerChart(...)

# Fig 2

In [605]:
df = (
    (
        pd.read_csv(
            "raw/Fig2.csv",
        ).dropna(how="all")
    )
    .set_index("Group")
    .stack()
    .reset_index()
)
df.columns = ["Group", "Variable", "Value"]

In [606]:
f = "fig2_balance"
f2 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f2.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f2 = df

In [607]:
base = (
    alt.Chart(f2)
    .encode(tooltip="tooltip:N")
    .transform_calculate(tooltip="datum.Variable+' 📈 '+round(datum.Value*100)+'%'")
)
selector1 = [
    "Liquid financial assets",
    "Illiquid financial assets",
    "Physical assets (incl. real estate)",
]
base1 = base.transform_filter(
    alt.FieldOneOfPredicate(field="Variable", oneOf=selector1)
)
bars1 = base1.mark_bar(height=15, opacity=0.9, color=colors["eco-turquiose"]).encode(
    x=alt.X(
        "Value:Q",
        stack="zero",
        axis=alt.Axis(
            grid=False,
            title="",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            format=".0%",
            tickCount=10,
            orient="bottom",
            labelAngle=0,
        ),
    ),
    order="Group:N",
    y=alt.Y(
        "Group:N",
        axis=alt.Axis(
            grid=False,
            title="Assets",
            titleX=0,
            titleY=-5,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
        ),
    ),
    color=alt.Color(
        "Variable:N",
        legend=alt.Legend(title=""),
        scale=alt.Scale(
            domain=selector1,
            range=[colors["eco-mid-blue"], colors["eco-turquiose"], colors["eco-red"]],
        ),
    ),
)
text1 = bars1.mark_text(dx=-2, dy=1, align="right").encode(
    text=alt.Text("Value:Q", format=".0%"),
    x=alt.X("Value:Q", stack="zero"),
    order="Group:N",
    color=alt.condition(
        datum.Value < 0.04,
        alt.ColorValue(None),
        alt.ColorValue("white"),
    ),
)
layer1 = (
    ((bars1 + text1).properties(height=alt.Step(20), width=300))
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + "a.json")
layer1

alt.LayerChart(...)

In [608]:
base = (
    alt.Chart(f2)
    .encode(tooltip="tooltip:N")
    .transform_calculate(tooltip="datum.Variable+' 📈 '+round(datum.Value*100)+'%'")
)
selector1 = [
    "Secured debt (incl. mortgages)",
    "Unsecured debt",
    "Net wealth (incl. real estate)",
]
base1 = base.transform_filter(
    alt.FieldOneOfPredicate(field="Variable", oneOf=selector1)
)
bars1 = base1.mark_bar(height=15, opacity=0.9, color=colors["eco-turquiose"]).encode(
    x=alt.X(
        "Value:Q",
        stack="zero",
        axis=alt.Axis(
            grid=False,
            title="",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            format=".0%",
            tickCount=10,
            orient="bottom",
            labelAngle=0,
        ),
    ),
    order="Group:N",
    y=alt.Y(
        "Group:N",
        axis=alt.Axis(
            grid=False,
            title="Liabilities",
            titleX=0,
            titleY=-5,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
        ),
    ),
    color=alt.Color(
        "Variable:N",
        legend=alt.Legend(title=""),
        scale=alt.Scale(
            domain=selector1,
            range=[colors["eco-mid-blue"], colors["eco-turquiose"], colors["eco-red"]],
        ),
    ),
)
text1 = bars1.mark_text(dx=-2, dy=1, align="right").encode(
    text=alt.Text("Value:Q", format=".0%"),
    x=alt.X("Value:Q", stack="zero"),
    order="Group:N",
    color=alt.condition(
        datum.Value < 0.04,
        alt.ColorValue(None),
        alt.ColorValue("white"),
    ),
)
layer2 = (
    ((bars1 + text1).properties(height=alt.Step(20), width=300))
    .configure_view(stroke=None)
    .properties(title="")
)
layer2.save("visualisation/" + f + "b.json")
layer2

alt.LayerChart(...)

In [609]:
base = (
    alt.Chart(f2)
    .encode(tooltip="tooltip:N")
    .transform_calculate(tooltip="datum.Variable+' 📈 '+round(datum.Value*100)+'%'")
)
selector1 = [
    "Food & Utilities",
    "Rent",
    "Debt repayment",
    "Utilities, transport, etc.",
    "Other expenses (incl. net savings)",
]
base1 = base.transform_filter(
    alt.FieldOneOfPredicate(field="Variable", oneOf=selector1)
)
bars1 = base1.mark_bar(height=15, opacity=0.9, color=colors["eco-turquiose"]).encode(
    x=alt.X(
        "Value:Q",
        stack="zero",
        axis=alt.Axis(
            grid=False,
            title="",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            format=".0%",
            tickCount=10,
            orient="bottom",
            labelAngle=0,
        ),
    ),
    order="Group:N",
    y=alt.Y(
        "Group:N",
        axis=alt.Axis(
            grid=False,
            title="Consumption",
            titleX=0,
            titleY=-5,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
        ),
    ),
    color=alt.Color(
        "Variable:N",
        legend=alt.Legend(title=""),
        scale=alt.Scale(
            domain=selector1,
            range=[
                colors["eco-mid-blue"],
                colors["eco-turquiose"],
                colors["eco-green"],
                colors["eco-red"],
                colors["eco-purple"],
            ],
        ),
    ),
)
text1 = bars1.mark_text(dx=-2, dy=1, align="right").encode(
    text=alt.Text("Value:Q", format=".0%"),
    x=alt.X("Value:Q", stack="zero"),
    order="Group:N",
    color=alt.condition(
        datum.Value < 0.05,
        alt.ColorValue(None),
        alt.ColorValue("white"),
    ),
)
layer3 = (
    ((bars1 + text1).properties(height=alt.Step(20), width=300))
    .configure_view(stroke=None)
    .properties(title="")
)
layer3.save("visualisation/" + f + "c.json")
layer3

alt.LayerChart(...)

# Fig 3

In [610]:
df = pd.read_csv(
    "raw/Fig3.csv",
).dropna(how="all")
df1 = df[df["Country"].isin(["Developed", "Emerging / E.Europe"])].sort_values(
    by="Fragility3m", ascending=True
)
df2 = df.iloc[[0]]
for c in df2.columns:
    df2[c] = np.nan
df2["Country"] = ""
df3 = df[~df["Country"].isin(["Developed", "Emerging / E.Europe"])].sort_values(
    by="Fragility3m", ascending=False
)
df = pd.concat([df1, df2, df3])
df.head()

<ipython-input-610-49e235858e53>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2[c] = np.nan
<ipython-input-610-49e235858e53>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["Country"] = ""


,Group,Fragility3m,Fragility6m,Fragility12m,Country
2,Developed,0.412736,0.547738,0.691076,Developed
1,Emerging / E.Europe,0.518751,0.633283,0.740711,Emerging / E.Europe
1,NaN,NaN,NaN,NaN,
27,Emerging / E.Europe,0.971597,0.986658,0.992845,South Africa
26,Emerging / E.Europe,0.824837,0.908119,0.964404,Latvia


In [611]:
f = "fig3_fragility"
f3 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f3.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f3 = df

In [612]:
base = (
    alt.Chart(f3)
    .encode(
        tooltip="tooltip:N",
        y=alt.Y(
            "Country:N",
            sort=[],
            axis=alt.Axis(
                grid=False,
                title="",
                titleX=0,
                titleY=-5,
                titleBaseline="bottom",
                titleAngle=0,
                titleAlign="left",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
            ),
        ),
    )
    .transform_calculate(
        tooltip="datum.Country+' 3️⃣ '+round(datum.Fragility3m*100)+'%'+' 6️⃣ '+round(datum.Fragility6m*100)+'%'+' 1️⃣2️⃣ '+round(datum.Fragility12m*100)+'%'"
    )
)
bars1 = base.mark_bar(height=15, opacity=0.9, color=colors["eco-mid-blue"]).encode(
    x=alt.X(
        "Fragility3m:Q",
        stack="zero",
        axis=alt.Axis(
            grid=False,
            title="",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            format=".0%",
            tickCount=10,
            orient="bottom",
            labelAngle=0,
        ),
    )
)
bars2 = base.mark_bar(height=11, opacity=0.9, color=colors["eco-light-blue"]).encode(
    x=alt.X("Fragility6m:Q", stack="zero")
)
bars3 = base.mark_bar(height=7, opacity=0.9, color=colors["eco-turquiose"]).encode(
    x=alt.X("Fragility12m:Q", stack="zero")
)
layer1 = (
    ((bars3 + bars2 + bars1).properties(height=alt.Step(20), width=300))
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + "a.json")
layer1

alt.LayerChart(...)

In [613]:
base = (
    alt.Chart(f3)
    .encode(
        tooltip="tooltip:N",
        y=alt.Y(
            "Country:N",
            sort=[],
            axis=alt.Axis(
                grid=False,
                title="",
                titleX=0,
                titleY=-5,
                titleBaseline="bottom",
                titleAngle=0,
                titleAlign="left",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
            ),
        ),
    )
    .transform_calculate(
        tooltip="datum.Country+' 3️⃣ '+round(datum.Fragility3m*100)+'%'+' 6️⃣ '+round(datum.Fragility6m*100)+'%'+' 1️⃣2️⃣ '+round(datum.Fragility12m*100)+'%'"
    )
)
bars1 = base.mark_circle(
    height=15, size=40, opacity=0.9, color=colors["eco-mid-blue"]
).encode(
    x=alt.X(
        "Fragility3m:Q",
        stack="zero",
        axis=alt.Axis(
            grid=False,
            title="",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            format=".0%",
            tickCount=10,
            orient="bottom",
            labelAngle=0,
        ),
    )
)
bars1l = base.mark_rule(
    height=11, strokeWidth=3, opacity=0.9, color=colors["eco-mid-blue"]
).encode(x=alt.X("Fragility3m:Q", stack="zero"))
bars2 = base.mark_circle(
    height=11, size=30, opacity=0.9, color=colors["eco-light-blue"]
).encode(x=alt.X("Fragility6m:Q", stack="zero"))
bars2l = base.mark_rule(
    height=11, strokeWidth=2, opacity=0.9, color=colors["eco-light-blue"]
).encode(x=alt.X("Fragility3m:Q", stack="zero"), x2="Fragility6m:Q")
bars3 = base.mark_circle(
    height=7, size=20, opacity=0.9, color=colors["eco-turquiose"]
).encode(x=alt.X("Fragility12m:Q", stack="zero"))
bars3l = base.mark_rule(
    height=11, strokeWidth=1, opacity=0.9, color=colors["eco-turquiose"]
).encode(x=alt.X("Fragility6m:Q", stack="zero"), x2="Fragility12m:Q")
labels1 = (
    bars1
    .mark_text(dy=13,color=colors['eco-mid-blue'])
    .encode(
        text="t:N"
    )
    .transform_calculate(t="'3'")
    .transform_filter("datum.Country=='Emerging / E.Europe'")
)
labels2 = (
    bars2
    .mark_text(dy=13,color=colors['eco-light-blue'])
    .encode(
        text="t:N"
    )
    .transform_calculate(t="'6'")
    .transform_filter("datum.Country=='Emerging / E.Europe'")
)
labels3 = (
    bars3
    .mark_text(dy=13,color=colors['eco-turquiose'],align='left',dx=-7)
    .encode(
        text="t:N"
    )
    .transform_calculate(t="'12 months'")
    .transform_filter("datum.Country=='Emerging / E.Europe'")
)
layer1 = (
    (
        (bars3 + bars3l + bars2 + bars2l + bars1 + bars1l+labels1+labels2+labels3).properties(
            height=alt.Step(14), width=300
        )
    )
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + "b.json")
layer1

alt.LayerChart(...)

# Fig 4

In [614]:
df1 = (
    pd.read_csv(
        "raw/Fig5-Developed.csv",
    ).dropna(how="all")
).drop("Unnamed: 0", axis=1)
df1["x"] = "developed"
df1.columns = ["months", "baseline", "liquidate", "x50", "debt", "developing"]
df2 = (
    pd.read_csv(
        "raw/Fig5-Emerging.csv",
    ).dropna(how="all")
).drop("Unnamed: 0", axis=1)
df2["x"] = "emerging"
df2.columns = ["months", "baseline", "liquidate", "x50", "debt", "developing"]
df = pd.concat([df1, df2])

In [615]:
f = "fig4_coping"
f4 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f4.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f4 = df
df.head()

,months,baseline,liquidate,x50,debt,developing
0,0.0,0.064168,0.045101,0.0,0.062125,developed
1,0.1,0.097863,0.070923,0.0,0.090390,developed
2,0.2,0.119375,0.087196,0.0,0.108352,developed
3,0.3,0.138751,0.102224,0.0,0.123902,developed
4,0.4,0.157681,0.116634,0.0,0.139728,developed


In [616]:
base = (
    alt.Chart(f4)
    .encode(
        x=alt.X(
            "months:Q",
            sort="ascending",
            axis=alt.Axis(
                grid=False,
                title="Months after initial shock",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                tickCount=12,
                titleFontWeight="normal",
                titleFontSize=10,
                orient="bottom",
                titleAnchor="end",
                titleY=-15,
                labelAngle=0,
            ),
        )
    )
    .transform_filter("datum.developing=='developed'")
)
line1 = base.mark_line(color=colors["eco-mid-blue"]).encode(
    y=alt.Y(
        "baseline:Q",
        sort=[],
        axis=alt.Axis(
            grid=True,
            title="Financial vulnerability (fraction of households)",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            labelAlign="left",
            ticks=False,
            labelBaseline="middle",
            labelPadding=-5,
            labelOffset=-10,
            titleX=30,
            titleY=-5,
            titleAngle=0,
            titleFontSize=10,
            titleFontWeight="normal",
            titleAlign="left",
            tickCount=8,
            format=".0%",
        ),
    )
)
line2 = base.mark_line(color=colors["eco-red"]).encode(y="x50:Q")
line3 = base.mark_line(color=colors["eco-light-blue"]).encode(y="debt:Q")
line4 = base.mark_line(color=colors["eco-turquiose"]).encode(y="liquidate:Q")
text1 = (
    line1.mark_text(color=colors["eco-mid-blue"], align="left", xOffset=5)
    .encode(text="t:N")
    .transform_calculate(t="'Baseline'")
    .transform_filter("datum.months==12")
)
text2 = (
    line2.mark_text(color=colors["eco-red"], align="left", xOffset=5)
    .encode(text="t:N")
    .transform_calculate(t="'50% income support'")
    .transform_filter("datum.months==12")
)
text3 = (
    line3.mark_text(color=colors["eco-light-blue"], align="left", xOffset=5)
    .encode(text="t:N")
    .transform_calculate(t="'Debt relief'")
    .transform_filter("datum.months==12")
)
text4 = (
    line4.mark_text(color=colors["eco-turquiose"], align="left", xOffset=5)
    .encode(text="t:N")
    .transform_calculate(t="'Liquidating illiquid financial assets'")
    .transform_filter("datum.months==12")
)
layer1 = (
    (
        (line1 + line2 + line3 + line4 + text1 + text2 + text3 + text4).properties(
            height=300, width=400
        )
    )
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + "a.json")
layer1

alt.LayerChart(...)

In [617]:
base = (
    alt.Chart(f4)
    .encode(
        x=alt.X(
            "months:Q",
            sort="ascending",
            axis=alt.Axis(
                grid=False,
                title="Months after initial shock",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                tickCount=12,
                titleFontWeight="normal",
                titleFontSize=10,
                orient="bottom",
                titleAnchor="end",
                titleY=-15,
                labelAngle=0,
            ),
        )
    )
    .transform_filter("datum.developing=='emerging'")
)
line1 = base.mark_line(color=colors["eco-mid-blue"]).encode(
    y=alt.Y(
        "baseline:Q",
        sort=[],
        axis=alt.Axis(
            grid=True,
            title="Financial vulnerability (fraction of households)",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            labelAlign="left",
            ticks=False,
            labelBaseline="middle",
            labelPadding=-5,
            labelOffset=-10,
            titleX=30,
            titleY=-5,
            titleAngle=0,
            titleFontSize=10,
            titleFontWeight="normal",
            titleAlign="left",
            tickCount=8,
            format=".0%",
        ),
    )
)
line2 = base.mark_line(color=colors["eco-red"]).encode(y="x50:Q")
line3 = base.mark_line(color=colors["eco-light-blue"]).encode(y="debt:Q")
line4 = base.mark_line(color=colors["eco-turquiose"]).encode(y="liquidate:Q")
text1 = (
    line1.mark_text(color=colors["eco-mid-blue"], align="left", xOffset=5, yOffset=-5)
    .encode(text="t:N")
    .transform_calculate(t="'Baseline'")
    .transform_filter("datum.months==12")
)
text2 = (
    line2.mark_text(color=colors["eco-red"], align="left", xOffset=5)
    .encode(text="t:N")
    .transform_calculate(t="'50% income support'")
    .transform_filter("datum.months==12")
)
text3 = (
    line3.mark_text(color=colors["eco-light-blue"], align="left", xOffset=5, yOffset=5)
    .encode(text="t:N")
    .transform_calculate(t="'Debt relief'")
    .transform_filter("datum.months==12")
)
text4 = (
    line4.mark_text(color=colors["eco-turquiose"], align="left", xOffset=5)
    .encode(text="t:N")
    .transform_calculate(t="'Liquidating illiquid financial assets'")
    .transform_filter("datum.months==12")
)
layer1 = (
    (
        (line1 + line2 + line3 + line4 + text1 + text2 + text3 + text4).properties(
            height=300, width=400
        )
    )
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + "b.json")
layer1

alt.LayerChart(...)